In [1]:
import pandas as pd
from tsnn.data_utils import *
from tsnn.models.lstnet import LSTNet
from tsnn.models.deepsense import DeepSense

Using TensorFlow backend.


In [2]:
batch_size = 64
sampling_step = 1
pred_delay = 24
pred_length = 1
nb_input_features = 321
input_cols = list(range(321))
timesteps_lstnet = 168
timesteps_deepsense = 1000
interest_vars = [315]

In [3]:
# Load data
raw_data = pd.read_csv('/Users/sofiene/Desktop/electricity.txt', header=None)

# Scale data
scaled_data, stats_df = scale_maxabs(raw_data)

# Split into inputs and targets
inputs, targets = inputs_targets_split(scaled_data, 
                                       input_cols=input_cols, 
                                       target_cols=interest_vars,
                                       samples_length=timesteps_lstnet, 
                                       pred_delay=pred_delay, 
                                       pred_length=pred_length)

# Train-Val-Test split
train_idx, val_idx, test_idx = train_val_split(targets, train_ratio=0.6, val_ratio=0.2)

In [7]:
# Create model
model = LSTNet(input_shape=(timesteps_lstnet, nb_input_features), 
               interest_vars=interest_vars)
model.compile(loss='mae', optimizer='adam')

In [8]:
# Prepare data generators
train_gen = sample_gen_rnn(inputs, targets, train_idx, timesteps_lstnet, sampling_step, batch_size, False)
val_gen = sample_gen_rnn(inputs, targets, val_idx, timesteps_lstnet, sampling_step, batch_size, False)

############### Temporary, function to be written ###############
tr_steps_per_epoch = (train_idx[1] - train_idx[0]) // batch_size
if (train_idx[1] - train_idx[0]) % batch_size != 0:
    tr_steps_per_epoch += 1
val_steps_per_epoch = (val_idx[1] - val_idx[0]) // batch_size
if (val_idx[1] - val_idx[0]) % batch_size != 0:
    val_steps_per_epoch += 1
#################################################################

In [9]:
# Train the model
model.fit_generator(generator=train_gen, 
                    steps_per_epoch=tr_steps_per_epoch, 
                    validation_data=val_gen, 
                    validation_steps=val_steps_per_epoch,
                    epochs=3, 
                    shuffle=False)

Epoch 1/3
245/245 [==============================] - 148s - loss: 0.1559 - val_loss: 0.1018

In [10]:
test_gen = sample_gen_rnn(inputs, targets, test_idx, timesteps_lstnet, sampling_step, batch_size, True)

############### Temporary, function to be written ###############
test_steps_per_epoch = (test_idx[1] - test_idx[0]) // batch_size
if (test_idx[1] - test_idx[0]) % batch_size != 0:
    test_steps_per_epoch += 1
#################################################################

In [11]:
preds = model.predict_generator(generator=test_gen, steps=test_steps_per_epoch, verbose=1)

preds.shape

82/82 [==============================] - 14s     

(5222, 1)

In [12]:
test_len = test_idx[1] - test_idx[0]
test_targets = targets.iloc[-test_len:]

test_targets.shape

(5222, 1)